In [5]:
import numpy as np
import glob
import os.path
import sys
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import cv2
import matplotlib.pyplot as plt
import imutils
import pandas as pd
import re
import datetime
from sklearn.decomposition import NMF
from sklearn.utils import shuffle
from skimage import draw
from skimage.io import imread
from scipy.sparse import csr_matrix,linalg
from sklearn.model_selection import train_test_split#import spams

import time
import math
from keras.models import *
from keras.layers import *
from keras.preprocessing import image
import keras.backend as K
import random
from sklearn.metrics import f1_score
from skimage.transform import resize
from skimage.filters import threshold_otsu
from tqdm import tqdm_notebook as tqdm 
#from tensorflow.keras.callbacks import TensorBoard
import keras
from tensorboard import version
from keras.models import model_from_json
from keras.layers.merge import concatenate, add
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)
config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction=0.2
config.gpu_options.allow_growth=True ##to use gpu as needed
config.log_device_placement = True  
sess = tf.Session(config=config)
set_session(sess)


 

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
!rm -rf /home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/input
!rm -rf /home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/mask
!rm -rf /home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/discard

In [6]:

MASK_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_mask'
IMAGE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_color_normalized'
IMAGE_WRITE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/input_normalized'
MASK_WRITE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/mask_normalized'
DISCARD_WRITE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/discard_normalized'
if not os.path.exists(IMAGE_WRITE_PATH):
    os.mkdir(IMAGE_WRITE_PATH)
    print('input directory made')
else:
    print('input directory exists')
    
if not os.path.exists(MASK_WRITE_PATH):
    os.mkdir(MASK_WRITE_PATH)
    print('mask directory made')
else:
    print('mask directory exists')
    
if not os.path.exists(DISCARD_WRITE_PATH):
    os.mkdir(DISCARD_WRITE_PATH)
    print('discard directory made')
else:
    print('discard directory exists')
    
image_list=os.listdir(IMAGE_PATH)

for image_ID in tqdm(image_list):
    # Format of image_ID : D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942.jpg
    image_name=os.path.join(IMAGE_PATH,image_ID)
    gray_img=cv2.imread(image_name,0)
    bgr_img=cv2.imread(image_name,1)
    mask_ID=image_ID.split('.')[0]+'_mask.'+image_ID.split('.')[-1]
    # Format of mask_ID: D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942_mask.jpg
    mask_name=os.path.join(MASK_PATH,mask_ID)
    mask_img=cv2.imread(mask_name,0)
    ret,mask_img=cv2.threshold(mask_img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    

    th,img_otsu=cv2.threshold(gray_img,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    temp_x,temp_y=np.where(img_otsu==255)

    for count_x,x in enumerate(range(img_otsu.shape[0]//512)):
        for count_y,y in enumerate(range(img_otsu.shape[1]//512)):
            
            temp_otsu=img_otsu[count_x*512:(count_x+1)*512,count_y*512:(count_y+1)*512]
            temp_mask=mask_img[count_x*512:(count_x+1)*512,count_y*512:(count_y+1)*512]
            
            
            if len(np.where(temp_otsu==0)[0])<0.6*len(np.where(temp_otsu==255)[0]) and len(np.where(temp_mask==255)[0])>=5000:
                img_write=bgr_img[count_x*512:(count_x+1)*512,count_y*512:(count_y+1)*512,:]
              
                write_path_image=os.path.join(IMAGE_WRITE_PATH,image_ID.split('.')[0]+\
                                          "_{}_{}.jpg".format(count_x,count_y))
        # Format of image_ID now: D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942_<count_x>_<count_y>.jpg
                cv2.imwrite(write_path_image,img_write)
                write_path_mask=os.path.join(MASK_WRITE_PATH,mask_ID.split('_mask')[0]+\
                                              "_{}_{}".format(count_x,count_y)+'_mask.jpg')
        # Format of mask_ID now: D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942_<count_x>_<count_y>_mask.jpg
                cv2.imwrite(write_path_mask,temp_mask)
            else:
                write_path_discard=os.path.join(DISCARD_WRITE_PATH,mask_ID.split('.')[0]+\
                                              "{}_{}.jpg".format(count_x,count_y))
                cv2.imwrite(write_path_discard,temp_mask)



input directory made
mask directory made
discard directory made


In [ ]:

input_unet=Input(shape=(None,None,3))
c1=Conv2D(32,(3,3),strides=(1,1),activation='relu', padding='same', name='conv1_1')(input_unet)
c2=Conv2D(32,(3,3),strides=(1,1),activation='relu', padding='same', name='conv1_2')(c1)
pool1=MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid')(c2)

c3=Conv2D(64,(3,3),strides=(1,1),activation='relu', padding='same', name='conv1_3')(pool1)
c4=Conv2D(64,(3,3),strides=(1,1),activation='relu', padding='same', name='conv1_4')(c3)
pool2=MaxPooling2D(pool_size=(2, 2), strides=None, padding='same')(c4)

c5=Conv2D(128,(3,3),strides=(1,1),activation='relu', padding='same', name='conv1_5')(pool2)
c6=Conv2D(128,(3,3),strides=(1,1),activation='relu', padding='same', name='conv1_6')(c5)
pool3=MaxPooling2D(pool_size=(2, 2), strides=None, padding='same')(c6)

c7=Conv2D(256,(3,3),strides=(1,1),activation='relu', padding='same', name='conv1_7')(pool3)
c8=Conv2D(256,(3,3),strides=(1,1),activation='relu', padding='same', name='conv1_8')(c7)
pool4=MaxPooling2D(pool_size=(2, 2), strides=None, padding='same')(c8)

c9=Conv2D(512,(3,3),strides=(1,1),activation='relu', padding='same', name='conv1_9')(pool4)
c10=Conv2D(512,(3,3),strides=(1,1),activation='relu', padding='same', name='conv1_10')(c9)

tc1=Conv2DTranspose(512, (3,3), strides=(2, 2), padding='same',name='deconv_1')(c10)
#tc1=Reshape(K.int_shape(c8))(tc1)
#print(K.int_shape(c8))
cat1 = concatenate([tc1, c8], axis=3)
c11=Conv2D(256,(3,3),strides=(1,1),activation='relu', padding='same', name='conv1_11')(cat1)
c12=Conv2D(256,(3,3),strides=(1,1),activation='relu', padding='same', name='conv1_12')(c11)

tc2=Conv2DTranspose(256, (3,3), strides=(2, 2), padding='same',name='deconv_2')(c12)
cat2 = concatenate([tc2, c6], axis=3)
c13=Conv2D(128,(3,3),strides=(1,1),activation='relu', padding='same', name='conv1_13')(cat2)
c14=Conv2D(128,(3,3),strides=(1,1),activation='relu', padding='same', name='conv1_14')(c13)

tc3=Conv2DTranspose(128, (3,3), strides=(2, 2), padding='same',name='deconv_3')(c14)
cat3 = concatenate([tc3, c4], axis=3)
c15=Conv2D(64,(3,3),strides=(1,1),activation='relu', padding='same', name='conv1_15')(cat3)
c16=Conv2D(64,(3,3),strides=(1,1),activation='relu', padding='same', name='conv1_16')(c15)

tc4=Conv2DTranspose(64, (3,3), strides=(2, 2), padding='same',name='deconv_4')(c16)
cat4 = concatenate([tc4, c2], axis=3)
c17=Conv2D(32,(3,3),strides=(1,1),activation='relu', padding='same', name='conv1_17')(cat4)
c18=Conv2D(32,(3,3),strides=(1,1),activation='relu', padding='same', name='conv1_18')(c17)

c19 = Conv2D(1, (1, 1),strides=(1,1),activation='relu', padding='same', name='conv1_19')(c18)




#OPT=keras.optimizers.SGD(lr=0.01,momentum=0.7,decay=0.00004,nesterov=True)

OPT=keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0000, amsgrad=False)

model_test=Model(inputs=[input_unet],outputs=[c19])
model_test.compile(optimizer=OPT, loss="binary_crossentropy", metrics=["accuracy"])
model_test.summary()
